주의점
1. 문장이 끊겨서 나눠져있음 (일상대화 데이터)<br>
`<SEP>` 토큰 관련

2. 라벨과 토큰 분리 관련<br>
봉구스 밥버거라는 --> '봉구스 밥버거' 로 라벨링되어있음

3. 비식별화 된 데이터 처리
%company name, name etc


"SDRW2000000026.1.1.97"



In [1]:
from transformers import BertModel, DistilBertModel
bert_model = BertModel.from_pretrained('monologg/kobert')

c:\Projects\AISec\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from tokenization_kobert import KoBertTokenizer
tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert') # monologg/distilkobert도 동일
tokenizer.tokenize("[CLS] 한국어 모델을 공유합니다. [SEP]")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


['[CLS]', '▁한국', '어', '▁모델', '을', '▁공유', '합니다', '.', '[SEP]']

In [10]:
print(tokenizer.tokenize("봉구스 밥버거라는 곳에서 근무를 했었을 때에도. [SEP]"))

['▁봉', '구', '스', '▁밥', '버', '거', '라는', '▁곳', '에서', '▁근무', '를', '▁했', '었', '을', '▁때', '에도', '.', '[SEP]']


In [11]:
import json

with open('./NIKL_NE_2022_v1.0/SXNE2202211218.json', 'r', encoding='utf-8') as f:
    test = json.load(f)

In [36]:
test['document'][0]['sentence'][5:11]

[{'id': 'SDRW2000000006.1.1.6',
  'form': '그래서 떡볶이',
  'word': [{'id': 1, 'form': '그래서', 'begin': 0, 'end': 3},
   {'id': 2, 'form': '떡볶이', 'begin': 4, 'end': 7}],
  'NE': [{'id': 1, 'form': '떡볶이', 'label': 'CV_FOOD', 'begin': 4, 'end': 7}]},
 {'id': 'SDRW2000000006.1.1.7',
  'form': '좋아하고',
  'word': [{'id': 1, 'form': '좋아하고', 'begin': 0, 'end': 4}],
  'NE': []},
 {'id': 'SDRW2000000006.1.1.8',
  'form': '카레도',
  'word': [{'id': 1, 'form': '카레도', 'begin': 0, 'end': 3}],
  'NE': [{'id': 1, 'form': '카레', 'label': 'CV_FOOD', 'begin': 0, 'end': 2}]},
 {'id': 'SDRW2000000006.1.1.9',
  'form': '매운 카레를 좋아하고',
  'word': [{'id': 1, 'form': '매운', 'begin': 0, 'end': 2},
   {'id': 2, 'form': '카레를', 'begin': 3, 'end': 6},
   {'id': 3, 'form': '좋아하고', 'begin': 7, 'end': 11}],
  'NE': [{'id': 1, 'form': '카레', 'label': 'CV_FOOD', 'begin': 3, 'end': 5}]},
 {'id': 'SDRW2000000006.1.1.10',
  'form': '돈가스를 먹어도',
  'word': [{'id': 1, 'form': '돈가스를', 'begin': 0, 'end': 4},
   {'id': 2, 'form': '먹어도', 'begi

In [50]:
from io import StringIO

sentences = []
labels = []
doc_sentences = test['document'][0]['sentence']

sent = StringIO()
label = []
for words in doc_sentences:
    # sentence
    sent.write(words['form'])

    # label
    ne = words['NE']
    words_length = len(words['word'])
    if not ne:
        label.extend(['O'] * words_length)  # add O label
    else:
        ne_iter = iter(ne)
        next_label = next(ne_iter)
        try:
            for i, word in enumerate(words['word'], 1):
                word_beg = word['begin']
                word_end = word['end']

                if word_beg <= next_label['begin'] and next_label['end'] <= word_end:  # label word
                    if word_beg == next_label['begin']:  # start of label
                        label.append(f"B-{next_label['label']}")

                    if word_beg < next_label['begin']:  # label is multiple words
                        label.append(f"I-{next_label['label']}")

                    if next_label['end'] <= word['end']:  # end of label
                        next_label = next(ne_iter)     
                
                else:
                    label.append('O')

        except StopIteration:
            label.extend(['O'] * (words_length - i))                

    # end of sentence
    if words['form'].endswith('.') or words['form'].endswith('?'):
        # separate . or ?
        sent.seek(sent.tell()-1)
        sent.write(f" {words['form'][-1]}")
        label.append('O')

        sentences.append(sent.getvalue())
        labels.append(' '.join(label))
        sent = StringIO()  # new sentence
        label = []
        
    else:
        sent.write(' ')

for s, l in zip(sentences, labels):
    print(s, l)

가장 좋아하는 음식이 무엇인가요 ? O O O O O
저는 주로 먹는 거를 좋아해서 음식을 가리지 않고 다 좋아하는데 매운 음식을 특히 좋아합니다 . O O O O O O O O O O O O O O O
그래서 떡볶이 좋아하고 카레도 매운 카레를 좋아하고 돈가스를 먹어도 매운 돈가스를 좋아합니다 . O B-CV_FOOD O B-CV_FOOD O B-CV_FOOD O B-CV_FOOD O O B-CV_FOOD O O
그래서 주로 매운 음식이면 다 좋아하는 것 같아요 . O O O O O O O O O
그러면 name1 님은 가장 좋아하는 음식이 무엇인가요 ? O O O O O O O O
저는 반대로 매운 걸 잘 먹지 못해서 매운 음식보다는 깔끔하고 담백한 음식을 좋아해요 . O O O O O O O O O O O O O O
그래서 주로 한식 중에서는 김치찌개 대신 된장찌개 같은 걸 더 좋아하고 요즘 유행하는 마라탕보다는 우동같이 맵지 않은 걸 더 좋아해요 . O O B-CV_FOOD_STYLE O B-CV_FOOD O B-CV_FOOD O O O O O O B-CV_FOOD B-CV_FOOD O O O O O O
그리고 어~ 간단하게 먹을 수 있는 라면류나 아니면 김밥 혹은 많은 사람들이 좋아하지만 저도 치킨을 좋아하고요 . O O O O O O O O B-CV_FOOD O O O O O B-CV_FOOD O O
그리고 햄버거나 아니면 스테이크 파스타류도 좋아합니다 . O B-CV_FOOD O B-CV_FOOD O O O
그럼 아까 별로 가리는 음식이 없다고 하셨는데 특히 이것만은 먹기 싫다 하는 음식이 있으신가요 ? O O O O O O O O O O O O O O O
저는 닭발은 정말 맵 매운 음식인데도 잘 못 먹거든요 . O B-CV_FOOD O O O O O O O O
그 이유가 보기에도 너무 징그럽고 입 안에 들어왔을 때에 느낌이 너무 이상해서 처음 먹었을 때 충격 받은 거 때문에 그 이후로 닭발을 한 번도 먹으려고 시도도 못 했던 것 같아요 . O O